# Algorytm przeszukiwania Tabu (TS)

## Biblioteki

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
from collections import deque
from openpyxl import load_workbook

## Pobór danych 

In [3]:
# file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

file_path1 = "C:/Users/wera6/Downloads/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/wera6/Downloads/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/wera6/Downloads/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaje ruchów

In [4]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [5]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [14]:
def tabu_search(distance_matrix, tabu_list_length=10, max_iterations=50, max_no_improve=1000, move_type="two_opt", time_limit=1000):
    start_time = time.time()
    
    n = len(distance_matrix)
    
    # initial solution
    best_solution = list(range(n))
    random.shuffle(best_solution)
    best_cost = calculate_path_cost(distance_matrix, best_solution)
    current_solution = best_solution[:]
    current_cost = best_cost
 
    tabu_list = deque(maxlen=tabu_list_length)
    no_improve = 0

    best_time = 0
 
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }
 
    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")
 
    move_generator = move_generators[move_type]
 
    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break
 
        neighborhood = list(move_generator(current_solution))
        neighborhood_costs = [calculate_path_cost(distance_matrix, neighbor) for neighbor in neighborhood]
 
        best_neighbor = None
        best_neighbor_cost = float('inf')
 
        for neighbor, cost in zip(neighborhood, neighborhood_costs):
            if (neighbor not in tabu_list) and (cost < best_neighbor_cost):
                best_neighbor = neighbor
                best_neighbor_cost = cost
 
        if best_neighbor is None:
            break
 
        # Update current solution and tabu list
        current_solution = best_neighbor
        current_cost = best_neighbor_cost
        tabu_list.append(current_solution)
        
        # Update best solution if improved
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_time = time.time() - start_time
            best_cost = current_cost
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= max_no_improve:
            print("Osiągnięto limit iteracji bez poprawy.")
            break

        if iteration % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f"Iteration {iteration}, Best Cost: {best_cost:.2f}, Elapsed Time: {elapsed_time:.2f} seconds")
 
    return best_solution, best_cost, best_time

## Generowanie wyników

Za podstawowe dane przyjmujemy:

* tabu_list_length = 10

* max_iterations = 10000

* max_no_improve = 50

* move_type = `two_opt`

* time_limit = 1000

Następnie badane są wpływy zmiany wartości poszczególnych parametrów 

Badanie wpływu zmian wartości parametru *move_type*

In [17]:
swap_p_1, swap_c_1, swap_t_1 = tabu_search(
    data1,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)

swap_p_2, swap_c_2, swap_t_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)

swap_p_3, swap_c_3, swap_t_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="swap",
    time_limit=1000
)

two_opt_p_1, two_opt_c_1, two_opt_t_1 = tabu_search(
    data1,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="two_opt",
    time_limit=1000
)

two_opt_p_2, two_opt_c_2, two_opt_t_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="two_opt",
    time_limit=1000
)

two_opt_p_3, two_opt_c_3, two_opt_t_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=100,
    move_type="two_opt",
    time_limit=1000
)

insertion_p_1, insertion_c_1, insertion_t_1 = tabu_search(
    data1,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)

insertion_p_2, insertion_c_2, insertion_t_2 = tabu_search(
    data2,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)

insertion_p_3, insertion_c_3, insertion_t_3 = tabu_search(
    data3,
    tabu_list_length=10,
    max_iterations=10000,
    max_no_improve=50,
    move_type="insertion",
    time_limit=1000
)


Iteration 0, Best Cost: 45043.00, Elapsed Time: 0.02 seconds
Iteration 10, Best Cost: 25745.00, Elapsed Time: 0.23 seconds
Iteration 20, Best Cost: 19815.00, Elapsed Time: 0.47 seconds
Iteration 30, Best Cost: 17222.00, Elapsed Time: 0.74 seconds
Iteration 40, Best Cost: 15750.00, Elapsed Time: 0.91 seconds
Iteration 50, Best Cost: 15419.00, Elapsed Time: 1.08 seconds
Iteration 60, Best Cost: 15376.00, Elapsed Time: 1.26 seconds
Iteration 70, Best Cost: 15360.00, Elapsed Time: 1.45 seconds
Iteration 80, Best Cost: 14813.00, Elapsed Time: 1.68 seconds
Iteration 90, Best Cost: 13396.00, Elapsed Time: 1.85 seconds
Iteration 100, Best Cost: 13363.00, Elapsed Time: 2.02 seconds
Iteration 110, Best Cost: 13363.00, Elapsed Time: 2.17 seconds
Iteration 120, Best Cost: 13363.00, Elapsed Time: 2.33 seconds
Iteration 130, Best Cost: 13363.00, Elapsed Time: 2.48 seconds
Iteration 140, Best Cost: 13363.00, Elapsed Time: 2.65 seconds
Osiągnięto limit iteracji bez poprawy.
Iteration 0, Best Cost: 539

KeyboardInterrupt: 

Tworzenie ramki danych dla TS

In [ ]:
W11 = {
        "Swap" : [swap_c_1, swap_c_2, swap_c_3],
        "Opt": [two_opt_c_1, two_opt_c_2, two_opt_c_3],
        "Insertion" : [insertion_c_1, insertion_c_2, insertion_c_3]
}

metody_TS = pd.DataFrame(W11)

Badanie wpływu zmian wartości parametru *tabu_list_length*

In [ ]:
tabu_tendures = [5, 10, 15, 20]

# DANE NR 1
results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})

tabu1 = pd.DataFrame(results)

# DANE NR 2
results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})

# Tworzenie ramki danych
tabu2 = pd.DataFrame(results)

# DANE NR 3
results = []
for tabu_list_length in tabu_tendures:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=tabu_list_length,
    )
    results.append({"PARAMETR": tabu_list_length, "WYNIK": best_cost})

# Tworzenie ramki danych
tabu3 = pd.DataFrame(results)

In [ ]:
merged_tabu = tabu1.merge(tabu2, on="PARAMETR").merge(tabu3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *max_iterations*

In [ ]:
max_iterations_values = [500, 1000, 1500, 2500]

# DANE NR 1 
results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})


max1 = pd.DataFrame(results)


# DANE NR 2

results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max2 = pd.DataFrame(results)


# DANE NR 3

results = []
for max_iterations in max_iterations_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=max_iterations,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost})

# Tworzenie ramki danych
max3 = pd.DataFrame(results)

In [ ]:
merged_max = max1.merge(max2, on="PARAMETR").merge(max3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *max_no_improve*

In [ ]:
max_no_improve_values = [10, 50, 150, 250]

# DANE NR 1 

results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})


max_no_improve1 = pd.DataFrame(results)


# DANE NR 2

results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})

# Tworzenie ramki danych
max_no_improve2 = pd.DataFrame(results)


# DANE NR 3

results = []
for max_no_improve in max_no_improve_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=max_no_improve,
        move_type="two_opt",
        time_limit=1000,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": max_no_improve, "WYNIK": best_cost})

# Tworzenie ramki danych
max_no_improve3 = pd.DataFrame(results)

In [ ]:
merged_max_no_improve = max_no_improve1.merge(max_no_improve2, on="PARAMETR").merge(max_no_improve3, on="PARAMETR")

Badanie wpływu zmian wartości parametru *time_limit*

In [ ]:
time_values = [60, 240, 500, 1000]
    
# DANE NR 1 

results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data1,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})


time1 = pd.DataFrame(results)


# DANE NR 2

results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data2,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})
# Tworzenie ramki danych
time2 = pd.DataFrame(results)


# DANE NR 3

results = []
for time_limit in time_values:
    _, best_cost = tabu_search(
        data3,
        max_iterations=10000,
        max_no_improve=50,
        move_type="two_opt",
        time_limit=time_limit,
        tabu_list_length=10,
    )
    results.append({"PARAMETR": time_limit, "WYNIK": best_cost})

# Tworzenie ramki danych
time3 = pd.DataFrame(results)

In [ ]:
merged_time = time1.merge(time2, on="PARAMETR").merge(time3, on="PARAMETR")

Zapis wyników do pliku

In [ ]:
result = {
    "Porównanie_metod": metody_TS,
    "tabu_list_length": merged_tabu,
    "max_iterations": merged_max,
    "max_no_improve": merged_max_no_improve,
    "time_limit": merged_time 
}

# Ścieżka do pliku Excel
file_name = "TS.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in result.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")